In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ensimag-wpf-2023/sample.csv
/kaggle/input/ensimag-wpf-2023/train.csv
/kaggle/input/ensimag-wpf-2023/test.csv


In [2]:
train=pd.read_csv('/kaggle/input/ensimag-wpf-2023/train.csv')
test=pd.read_csv('/kaggle/input/ensimag-wpf-2023/test.csv')
sub=pd.read_csv('/kaggle/input/ensimag-wpf-2023/sample.csv')

In [3]:
train

,date,u10,v10,u100,v100,production
0,2020-01-01 01:00:00,2.124600,-2.681966,2.864280,-3.666076,0.000000
1,2020-01-01 02:00:00,2.521695,-1.796960,3.344859,-2.464761,0.054879
2,2020-01-01 03:00:00,2.672210,-0.822516,3.508448,-1.214093,0.110234
3,2020-01-01 04:00:00,2.457504,-0.143642,3.215233,-0.355546,0.165116
4,2020-01-01 05:00:00,2.245898,0.389576,2.957678,0.332701,0.156940
...,...,...,...,...,...,...
8778,2020-12-31 19:00:00,1.201009,0.682997,2.862055,1.878258,0.065501
8779,2020-12-31 20:00:00,1.917654,1.455812,2.825571,2.288703,0.092285
8780,2020-12-31 21:00:00,2.192580,1.544936,3.010909,2.230068,0.042477
8781,2020-12-31 22:00:00,2.416432,1.232599,3.255107,1.724829,0.104407


In [4]:
test

,date,u10,v10,u100,v100
0,2021-01-01 00:00:00,3.103149,1.295629,4.146771,1.672058
1,2021-01-01 01:00:00,3.456026,1.799494,4.624995,2.329869
2,2021-01-01 02:00:00,3.716234,1.893639,4.987532,2.435546
3,2021-01-01 03:00:00,4.029650,2.195310,5.442522,2.855735
4,2021-01-01 04:00:00,4.200922,2.402018,5.703258,3.147100
...,...,...,...,...,...
7977,2021-11-29 20:00:00,-2.014530,1.793392,-4.502124,3.949871
7978,2021-11-29 21:00:00,-3.366064,2.831022,-4.991581,4.218858
7979,2021-11-29 22:00:00,-3.433361,2.457599,-4.796116,3.407203
7980,2021-11-29 23:00:00,-3.224261,2.049055,-4.407834,2.755195


In [5]:
sub

,date,production
0,2021-01-01 00:00:00,0.25
1,2021-01-01 01:00:00,0.25
2,2021-01-01 02:00:00,0.25
3,2021-01-01 03:00:00,0.25
4,2021-01-01 04:00:00,0.25
...,...,...
7977,2021-11-29 20:00:00,0.25
7978,2021-11-29 21:00:00,0.25
7979,2021-11-29 22:00:00,0.25
7980,2021-11-29 23:00:00,0.25


In [6]:
train=train.drop('date',axis=1)
test=test.drop('date',axis=1)

In [7]:
X=train.drop('production',axis=1)
y=train['production']

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.base import clone

# Instantiate the models
rf_reg = RandomForestRegressor(random_state=42)
gb_reg = GradientBoostingRegressor(random_state=42)
cat_reg = CatBoostRegressor(random_state=42, verbose=0)
lgbm_reg = LGBMRegressor(random_state=42)

# Train the models
rf_reg.fit(X_train, y_train)
gb_reg.fit(X_train, y_train)
cat_reg.fit(X_train, y_train)
lgbm_reg.fit(X_train, y_train)

# Make predictions on the test set
rf_pred = rf_reg.predict(X_test)
gb_pred = gb_reg.predict(X_test)
cat_pred = cat_reg.predict(X_test)
lgbm_pred = lgbm_reg.predict(X_test)

# Calculate the mean squared error for each model
rf_mse = mean_squared_error(y_test, rf_pred)
gb_mse = mean_squared_error(y_test, gb_pred)
cat_mse = mean_squared_error(y_test, cat_pred)
lgbm_mse = mean_squared_error(y_test, lgbm_pred)

print("Random Forest Regressor MSE: {:.4f}".format(rf_mse))
print("Gradient Boosting Regressor MSE: {:.4f}".format(gb_mse))
print("CatBoost Regressor MSE: {:.4f}".format(cat_mse))
print("LightGBM Regressor MSE: {:.4f}".format(lgbm_mse))

# Create an ensemble of the four models
ensemble = [clone(rf_reg), clone(gb_reg), clone(cat_reg), clone(lgbm_reg)]
for model in ensemble:
    model.fit(X_train, y_train)

# Make predictions using the ensemble
ensemble_pred = [model.predict(X_test) for model in ensemble]
ensemble_pred = sum(ensemble_pred) / len(ensemble_pred)

# Calculate the mean squared error for the ensemble
ensemble_mse = mean_squared_error(y_test, ensemble_pred)
print("Ensemble MSE: {:.4f}".format(ensemble_mse))



Random Forest Regressor MSE: 0.0330
Gradient Boosting Regressor MSE: 0.0339
CatBoost Regressor MSE: 0.0314
LightGBM Regressor MSE: 0.0327
Ensemble MSE: 0.0316


In [13]:
# y_pred=cat_reg.predict(test)
y_pred = [model.predict(test) for model in ensemble]
y_pred = sum(ensemble_pred) / len(ensemble_pred)

In [14]:
df=pd.read_csv('/kaggle/input/ensimag-wpf-2023/test.csv')
df['production']=y_pred
od=df[['date','production']]
od

,date,production
0,2021-01-01 00:00:00,0.305815
1,2021-01-01 01:00:00,0.305815
2,2021-01-01 02:00:00,0.305815
3,2021-01-01 03:00:00,0.305815
4,2021-01-01 04:00:00,0.305815
...,...,...
7977,2021-11-29 20:00:00,0.305815
7978,2021-11-29 21:00:00,0.305815
7979,2021-11-29 22:00:00,0.305815
7980,2021-11-29 23:00:00,0.305815


In [15]:
od.to_csv('submission1.csv',index=False)

In [17]:
import joblib

# Save the pre-trained model
joblib.dump(rf_reg, 'model.pkl')

['model.pkl']